In [39]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-question-pairs/train.csv.zip
/kaggle/input/quora-question-pairs/sample_submission.csv.zip
/kaggle/input/quora-question-pairs/test.csv
/kaggle/input/quora-question-pairs/test.csv.zip


In [40]:
import re
!pip install --user distance
import distance
import plotly.express as px
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from fuzzywuzzy import fuzz
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [77]:
df = pd.read_csv("/kaggle/input/quora-question-pairs/train.csv.zip")

# **A. Exploratory Data Analysis (EDA)**

In [78]:
df.shape

(404290, 6)

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404290 non-null  int64 
 1   qid1          404290 non-null  int64 
 2   qid2          404290 non-null  int64 
 3   question1     404289 non-null  object
 4   question2     404288 non-null  object
 5   is_duplicate  404290 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [43]:
df.isnull().sum()

id              0
qid1            0
qid2            0
question1       1
question2       2
is_duplicate    0
dtype: int64

In [44]:
# Removing the rows containing NaN values. 
df.dropna(inplace=True)

In [45]:
df.duplicated().sum()

0

In [46]:
print("Number of LABEL entries representing duplicate and unique values:")
print(df['is_duplicate'].value_counts())

print("\nPercentage of LABEL entries representing duplicate and unique values:")
print((df['is_duplicate'].value_counts()/df['is_duplicate'].count())*100)

Number of LABEL entries representing duplicate and unique values:
0    255024
1    149263
Name: is_duplicate, dtype: int64

Percentage of LABEL entries representing duplicate and unique values:
0    63.079941
1    36.920059
Name: is_duplicate, dtype: float64


In [79]:
fig = px.bar(df['is_duplicate'].value_counts())
fig.show()

In [48]:
qid = pd.Series(df['qid1'].tolist() + df['qid2'].tolist())
print('Total number of questions: ', qid.shape[0])
print('Number of unique questions: ', np.unique(qid).shape[0])
print('Number of duplicate questions: ', qid.value_counts()[qid.value_counts()>1].shape[0])

Total number of questions:  808574
Number of unique questions:  537929
Number of duplicate questions:  111778


In [49]:
df.drop(['qid1', 'qid2'], axis=1, inplace=True)

# **B. Data Preprocessing**

In [50]:
df['lengthQ1'] = df['question1'].str.len() 
df['lengthQ2'] = df['question2'].str.len()

In [51]:
def preprocessing(question):
    question = str(question).lower().strip()
    question = question.replace('%', ' percent')
    question = question.replace('$', ' dollar ')
    question = question.replace('₹', ' rupee ')
    question = question.replace('€', ' euro ')
    question = question.replace('@', ' at ')
    
    
    question = question.replace(',000,000,000 ', 'b ')
    question = question.replace(',000,000 ', 'm ')
    question = question.replace(',000 ', 'k ')
    question = re.sub(r'([0-9]+)000000000', r'\1b', question)
    question = re.sub(r'([0-9]+)000000', r'\1m', question)
    question = re.sub(r'([0-9]+)000', r'\1k', question)
    
    contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    decontractedQuestion = []

    for word in question.split():
        if word in contractions:
            word = contractions[word]
        decontractedQuestion.append(word)

    question = ' '.join(decontractedQuestion)
    question = question.replace("'ve", " have")
    question = question.replace("n't", " not")
    question = question.replace("'re", " are")
    question = question.replace("'ll", " will")
    
    question = BeautifulSoup(question)
    question = question.get_text()
    
    pattern = re.compile('\W')
    question = re.sub(pattern, ' ', question).strip()
    
    return question

In [52]:
df['question1'] = df['question1'].apply(preprocessing)
df['question2'] = df['question2'].apply(preprocessing)

/opt/conda/lib/python3.7/site-packages/bs4/__init__.py:439: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning


# **C. Feature Engineering**

## **Evaluating Basic Features**

In [53]:
df['numWordsQ1'] = df['question1'].apply(lambda row: len(row.split(" ")))
df['numWordsQ2'] = df['question2'].apply(lambda row: len(row.split(" ")))

In [54]:
def totalWordsCounter(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return (len(w1) + len(w2))

In [55]:
df['numTotalWords'] = df.apply(totalWordsCounter, axis=1)

In [56]:
def commonWordsCounter(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return len(w1 & w2)

In [57]:
df['numCommonWords'] = df.apply(commonWordsCounter, axis=1)

In [58]:
df['commonWordRatio'] = round(df['numCommonWords']/df['numTotalWords'],2)

## **Evaluating Token Features**

In [59]:
def evaluateTokenFeatures(row):
    
    question1 = row['question1']
    question2 = row['question2']
    
    SAFE_DIV = 0.0001 
    STOP_WORDS = stopwords.words("english")
    tokenFeatures = [0.0]*6
    
    tokensQ1 = question1.split()
    tokensQ2 = question2.split()
    
    if len(tokensQ1) == 0 or len(tokensQ2) == 0:
        return tokenFeatures
    
    stopsWordsQ1 = set([word for word in tokensQ1 if word in STOP_WORDS])
    stopsWordsQ2 = set([word for word in tokensQ2 if word in STOP_WORDS])
    
    numCommonStopWords = len(stopsWordsQ1.intersection(stopsWordsQ2))
    numCommonTokens = len(set(tokensQ1).intersection(set(tokensQ2)))
    
    tokenFeatures[0] = numCommonStopWords / (min(len(stopsWordsQ1), len(stopsWordsQ2)) + SAFE_DIV)
    tokenFeatures[1] = numCommonStopWords / (max(len(stopsWordsQ1), len(stopsWordsQ2)) + SAFE_DIV)
    tokenFeatures[2] = numCommonTokens / (min(len(tokensQ1), len(tokensQ2)) + SAFE_DIV)
    tokenFeatures[3] = numCommonTokens / (max(len(tokensQ1), len(tokensQ2)) + SAFE_DIV)
    
    tokenFeatures[4] = int(tokensQ1[-1] == tokensQ2[-1])
    tokenFeatures[5] = int(tokensQ1[0] == tokensQ2[0])
    
    return tokenFeatures

In [60]:
tokenFeatures = df.apply(evaluateTokenFeatures, axis=1)

df["numCommonStopWordsMin"] = list(map(lambda x: x[0], tokenFeatures))
df["numCommonStopWordsMax"] = list(map(lambda x: x[1], tokenFeatures))
df["numCommonTokensMin"] = list(map(lambda x: x[2], tokenFeatures))
df["numCommonTokensMax"] = list(map(lambda x: x[3], tokenFeatures))
df["isLastWordSame"]  = list(map(lambda x: x[4], tokenFeatures))
df["isFirstWordSame"] = list(map(lambda x: x[5], tokenFeatures))

In [61]:
def evaluateLengthFeatures(row):
    
    question1 = row['question1']
    question2 = row['question2']
    
    lengthFeatures = [0.0]*2
    
    tokensQ1 = question1.split()
    tokensQ2 = question2.split()
    
    if len(tokensQ1) == 0 or len(tokensQ2) == 0:
        return lengthFeatures
    
    lengthFeatures[0] = abs(len(tokensQ1) - len(tokensQ2))
    
    substrings = list(distance.lcsubstrings(question1, question2))
    lengthFeatures[1] = len(substrings[0]) / (min(len(question1), len(question2)) + 1)
    
    return lengthFeatures

In [62]:
#lengthFeatures = df.apply(evaluateLengthFeatures, axis=1)

#df['absLengthDifference'] = list(map(lambda x: x[0], lengthFeatures))
#df['longestSubstringRatio'] = list(map(lambda x: x[1], lengthFeatures))

## **Evaluating Fuzzy Features**

In [63]:
def evaluateFuzzyFeatures(row):
    
    question1 = row['question1']
    question2 = row['question2']
    
    fuzzyFeatures = [0.0]*4
    
    fuzzyFeatures[0] = fuzz.QRatio(question1, question2)
    fuzzyFeatures[1] = fuzz.partial_ratio(question1, question2)
    fuzzyFeatures[2] = fuzz.token_sort_ratio(question1, question2)
    fuzzyFeatures[3] = fuzz.token_set_ratio(question1, question2)

    return fuzzyFeatures

In [64]:
fuzzyFeatures = df.apply(evaluateFuzzyFeatures, axis=1)

df['fuzzRatio'] = list(map(lambda x: x[0], fuzzyFeatures))
df['fuzzPartialRatio'] = list(map(lambda x: x[1], fuzzyFeatures))
df['tokenSortRatio'] = list(map(lambda x: x[2], fuzzyFeatures))
df['tokenSetRatio'] = list(map(lambda x: x[3], fuzzyFeatures))

# **D. Dimentionality reduction**

In [65]:
X = MinMaxScaler().fit_transform(df[['numCommonStopWordsMin', 'numCommonStopWordsMax', 'numCommonTokensMin', 'numCommonTokensMax', 'isLastWordSame', 'isFirstWordSame', 'fuzzRatio', 'fuzzPartialRatio' , 'tokenSortRatio' , 'tokenSetRatio']])
Y = df['is_duplicate'].values

In [66]:
#tSNE2d = TSNE(n_components=2, init='random', random_state=101, method='barnes_hut', n_iter=1000, verbose=2, angle=0.5).fit_transform(X)

In [67]:
pd.set_option('display.max_columns', None)
df.head(5)

,id,question1,question2,is_duplicate,lengthQ1,lengthQ2,numWordsQ1,numWordsQ2,numTotalWords,numCommonWords,commonWordRatio,numCommonStopWordsMin,numCommonStopWordsMax,numCommonTokensMin,numCommonTokensMax,isLastWordSame,isFirstWordSame,fuzzRatio,fuzzPartialRatio,tokenSortRatio,tokenSetRatio
0,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,66,57,14,12,23,11,0.48,0.999983,0.999983,0.916659,0.785709,0.0,1.0,93,100,93,100
1,1,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,51,88,12,17,26,8,0.31,0.749981,0.599988,0.699993,0.466664,0.0,1.0,66,74,63,86
2,2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,73,59,14,10,24,4,0.17,0.399992,0.249997,0.399996,0.285712,0.0,1.0,54,53,66,66
3,3,why am i mentally very lonely how can i solve it,find the remainder when math 23 24 math i...,0,50,65,12,18,22,1,0.05,0.000000,0.000000,0.000000,0.000000,0.0,0.0,35,39,36,36
4,4,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0,76,39,15,7,21,4,0.19,0.999950,0.666644,0.571420,0.307690,0.0,1.0,46,55,47,67


# **E. Vectorization**

In [68]:
questionDataFrame = df[['question1','question2']]
usefulDataFrame = df.drop(columns=['id', 'question1','question2'])
questions = list(questionDataFrame['question1']) + list(questionDataFrame['question2'])

In [69]:
cv = CountVectorizer(max_features=500)
arrayQ1, arrayQ2 = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [70]:

temp_df1 = pd.DataFrame(arrayQ1, index=questionDataFrame.index)
temp_df2 = pd.DataFrame(arrayQ2, index=questionDataFrame.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)

In [71]:
temp_df.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

# **F. Model Validation & Training**

In [72]:
finalDataFrame = pd.concat([usefulDataFrame, temp_df], axis=1)

In [73]:
X_train, X_test, Y_train, Y_test = train_test_split(finalDataFrame.iloc[:, 1:].values, finalDataFrame.iloc[:,0].values, test_size=0.2, random_state=1)

In [74]:
rf = RandomForestClassifier()
rf.fit(X_train,Y_train)

RandomForestClassifier()

# **G. Prediction Results**

In [75]:
Y_pred = rf.predict(X_test)
accuracy_score(Y_test,Y_pred)

0.8231838531747013

In [76]:
confusion_matrix(Y_test, Y_pred)

array([[44474,  6564],
       [ 7733, 22087]])